In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dotenv
import sagemaker
import boto3
import os
from datasets import load_dataset, load_from_disk

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nasoungadoy/Library/Application Support/sagemaker/config.yaml


In [3]:
dotenv.load_dotenv()

True

In [4]:
boto3.setup_default_session(profile_name="ai-sandbox-sso")
role_name = os.environ["SAGEMAKER_ROLE_NAME"]

In [8]:
MODEL_RUN_S3_OUTPUT = "s3://sagemaker-us-east-1-542301657622/mbay-nmt-google-mt5-small-2023-10-15-14-26-51-101/output/"

In [9]:
from sagemaker.s3 import S3Downloader

In [5]:
local_path = "../models/mbay-nmt-google-mt5-xl-2023-10-22-10-48-02-749/"

In [10]:

os.makedirs(local_path, exist_ok=True)

# download model from S3
S3Downloader.download(
    s3_uri=MODEL_RUN_S3_OUTPUT,  # s3 uri where the trained model is located
    local_path=local_path,  # local path where *.tar.gz will be saved
    # sagemaker_session=sess # sagemaker session used for training the model
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nasoungadoy/Library/Application Support/sagemaker/config.yaml


['models/mbay-nmt-google-mt5-small/model.tar.gz']

In [6]:
# unzip model
import tarfile


tar = tarfile.open(f"{local_path}/model.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()
os.remove(f"{local_path}/model.tar.gz")

In [7]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

/Users/nasoungadoy/Code/mbay-translator/py/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [8]:
from transformers import pipeline

translator = pipeline("translation", model=local_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/nasoungadoy/Code/mbay-translator/py/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
translator("Translate French to Mbay: Women.", max_new_tokens=256)

[{'translation_text': '<extra_id_0> a very very very very poor <extra_id_1>. <extra_id_2>. <extra_id_3>. <extra_id_4>. <extra_id_5>. <extra_id_6>. <extra_id_7>. <extra_id_8>. Men <extra_id_9>. Men <extra_id_10>. Men <extra_id_11>. Men <extra_id_12>. Women <extra_id_13>. Men <extra_id_14>. Men <extra_id_15>. Men <extra_id_16>. Women. Women <extra_id_17>. Men <extra_id_18>. Women. Women. Women <extra_id_19>. Women <extra_id_20>. Women <extra_id_21>. Women. Women. Women <extra_id_22>. Women <extra_id_23>. Women. Women <extra_id_24>. Women <extra_id_25>. Women. Women. Women <extra_id_26>. Women <extra_id_27>. Women. Women. Women <extra_id_28>. Women <extra_id_29>. Women. Women <extra_id_30>. Women. Women. Women <extra_id_31>. Women. Women <extra_id_32>. Women. Women. Women. Women. Women <extra_id_33>. Women <extra_id_34>. Women <extra_id_35>. Women. Women <extra_id_36>. Women <extra_id_37>. Women. Women <extra_id_38>. Women <extra_id_39>. Women. Women <extra_id_40>. Women <extra_id_41>. Wo

In [25]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
else:
    mps_device = None
print(mps_device)

mps


In [49]:
model = AutoModelForSeq2SeqLM.from_pretrained(local_path)
tokenizer = AutoTokenizer.from_pretrained(local_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
model = translator.model
tokenizer = translator.tokenizer

In [20]:
"".join(tokenizer.batch_decode([408, 3928, 264, 1135, 264, 48178, 4345, 265, 264, 118592, 8740, 1]))

'kété-ì-ɓēe-gɨ̄</s>'

In [18]:
tokenizer("kété-ì-ɓēe-gɨ̄")

{'input_ids': [408, 3928, 264, 1135, 264, 48178, 4345, 265, 264, 118592, 8740, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
text = "Translate Mbay to Mbay: Women."

encoded_input = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(**encoded_input)
output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [47]:
# model = AutoModelForSeq2SeqLM.from_pretrained(local_path, device_map=mps_device)
# tokenizer = AutoTokenizer.from_pretrained(local_path, device_map=mps_device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
# text = "Translate English to Mbay: She is cooking the soup."

# encoded_input = tokenizer(text, return_tensors="pt").input_ids.to(mps_device)
# generated_tokens = model.generate(input_ids=encoded_input)
# output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [17]:
output

['<extra_id_0> a traditional Telugu language. Telugu <extra_id_1>. Telugu <extra_id_2>. Telugu']